# Data import

In [1]:
## Import packages we will use 
import json
import pprint 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import collections 
import tensorflow as tf
from sklearn.manifold import TSNE

import json
import yaml  # https://pypi.python.org/pypi/PyYAML
from boltons.iterutils import remap  # https://pypi.python.org/pypi/boltons

#http://sedimental.org/remap.html#

In [2]:
# data = []
# i=0
# with open('products.json','r') as f:
#     for line in f:
#         i=i+1
#         if (i%10000==0):
#             print(i)
#         line = json.loads(line)
#         variant_id = line['variant_id'].replace(line['product_id'],"")
#         if variant_id != '':
#             line['index_missing'] = True
#             index = int(variant_id)-1
#             varianty = line['variant_data']
#             if len(varianty)>index:
#                 line['variant_data'] = line['variant_data'][index]
#                 line['index_out_of_range'] = False
#             else:
#                 line['variant_data'] = line['variant_data'][-1]
#                 line['index_out_of_range'] = True
#         else:
#             line['variant_data'] = line['variant_data'][0]
#             line['index_out_of_range'] = False
#             line['index_missing'] = True
#         data.append(line)

In [3]:
# def visit(path, key, value):
#     if key == 'date':
#         return key, pd.to_datetime(value)
#     return key, value
# data = remap(data, visit=visit)

In [4]:
# def flatten_json(y):
#     out = {}

#     def flatten(x, name=''):
#         if type(x) is dict:
#             for a in x:
#                 flatten(x[a], name + a + '_')
#         elif type(x) is list:
#             i = 0
#             for a in x:
#                 flatten(a, name + str(i) + '_')
#                 i += 1
#         else:
#             out[name[:-1]] = x

#     flatten(y)
#     return out

In [5]:
# def visit(path, key, value):
#     if key == 'params':
#         li=[]
#         for param in value:
#             l = list(param['values'].values())
#             if not isinstance(l[0], str):
#                 l = [item for sublist in l for item in sublist]
#             l = li.append((param['id'],l[0]))
#         return key, dict(li)
#     return key, value
# data2 = remap(data, visit=visit)

In [6]:
# data3=[]
# for line in data2:
#     line=flatten_json(line)
#     data3.append(line)

In [7]:
# df = pd.DataFrame(data3)

In [8]:
# co nema vubec nic?
# i=0
# for line in data2:
#     if line['variant_data']['params']=={}:
#         i=i+1
# print(i)

In [9]:
# df.to_pickle('data_frame.pkl')

## Data import zparsovanych dat

In [10]:
pd.options.display.max_rows=999
df=pd.read_pickle('data_frame.pkl')
df['date']=pd.to_datetime(df['date'])
n=df.shape[0]

## Sloupce podle vyskytu

In [12]:
#df.count().sort_values(ascending=False)/n

In [131]:
cols= ['brand_brand_id', 'date',
       'index_missing', 'index_out_of_range', 'product_id', 'product_type_id',
       'product_type_title_C', 'title_full_C', 'user_id',
       'variant_data_params_AUTUMN_SHOES', 'variant_data_params_BOYS_SHOES',
       'variant_data_params_COLLECTION_NM', 'variant_data_params_COLOR',
       'variant_data_params_DETERMINATION',
       'variant_data_params_DETERMINATION_OF_SHOES',
       'variant_data_params_EXTENDED_SELECTION', 'variant_data_params_FASHION',
       'variant_data_params_GIRLS_SHOES', 'variant_data_params_HEEL',
       'variant_data_params_MATERIAL', 'variant_data_params_MEMBRANE',
       'variant_data_params_MENS_SHOES', 'variant_data_params_MEN_WOMEN',
       'variant_data_params_MODEL_OF_YEAR', 'variant_data_params_RUNNER',
       'variant_data_params_SEASON_OF_YEAR', 'variant_data_params_SHOES_HIGH',
       'variant_data_params_SIZE', 'variant_data_params_SIZE_EUR',
       'variant_data_params_SIZE_UK', 'variant_data_params_SPORT',
       'variant_data_params_SPRING_SHOES', 'variant_data_params_SUMMER_SHOES',
       'variant_data_params_TIE', 'variant_data_params_TYPE_OF_BINDING',
       'variant_data_params_TYPE_OF_SHOES', 'variant_data_params_TYPE_OF_TOE',
       'variant_data_params_TYPE_USAGE', 'variant_data_params_WEIGHT_G',
       'variant_data_params_WINTER_SHOES', 'variant_data_params_WOMENS_SHOES',
       'variant_data_price_CZ1000_current_price',
       'variant_data_price_CZ1000_price',
       'variant_data_price_CZ1000_promotion_price',
       'variant_data_price_CZ1000_rrp',
       'variant_id']
df=df.loc[:,cols]
nice_users = ((df.groupby(['user_id'])['date'].count())>3) &((dff.groupby(['user_id'])['date'].count())<100)

cols_to_use = ['date',
               'variant_data_params_SIZE_EUR',
               'product_id',
               'user_id',
               'title_full_C',
               'variant_id',
               'variant_data_params_COLOR'
              ]
df = df[cols_to_use].copy()

## Window funkce na identifikování návštěv

jak rozseknout dvě návštevy je nastavitelný parametr, momentálně to dávám jako pevné číslo, ale šlo by to i líp

In [143]:
def ff(x):
    x=x.sort_values('date')
    x['date']=pd.to_numeric(x['date'])
    x['time_diff']=pd.to_timedelta(x['date'].rolling(window=2,center=False).apply(lambda x: x[1] - x[0]))
    x['date']=pd.to_datetime(x['date'])
    x['is_new']=x['time_diff']>pd.to_timedelta(1800,unit='s')
    k=[]
    i=0
    for j in x.is_new:
        if j:
            i=i+1
        k.append(i)
    x['visit_number']=k
    navstevy = x.groupby('visit_number')['date'].count()
    return [i+1,navstevy.max(),navstevy.min(),navstevy.median()]

In [144]:
df_grouped=df.groupby('user_id').apply(ff)
df_visits = pd.DataFrame(df_grouped.values.tolist(), columns = [
    'number_of_visits',
    'maximal_clicks',
    'minimal_clicks',
    'median_clicks'
])
df_visits.index=df_grouped.index

#### Nejake statistiky

In [136]:
df_visits.loc[lambda x:x.number_of_visits>4].shape

(55, 3)

## funkce na identifikování sessions

Momentálně dělám jen podle toho jaký je mezi nimi rozdíl, šlo by daleko líp


In [145]:
def ff(x):
    x=x.sort_values('date')
    x['date']=pd.to_numeric(x['date'])
    x['time_diff']=pd.to_timedelta(x['date'].rolling(window=2,center=False).apply(lambda x: x[1] - x[0]))
    x['date']=pd.to_datetime(x['date'])
    x['is_new']=x['time_diff']>pd.to_timedelta(24*3600,unit='s')
    k=[]
    i=0
    for j in x.is_new:
        if j:
            i=i+1
        k.append(i)
    x['visit_number']=k
    navstevy = x.groupby('visit_number')['date'].count()
    return [i+1,navstevy.max(),navstevy.min(),navstevy.median()]

In [146]:
df_grouped=df.groupby('user_id').apply(ff)
df_sessions = pd.DataFrame(df_grouped.values.tolist(), columns = [
    'number_of_visits',
    'maximal_clicks',
    'minimal_clicks',
    'median_clicks'
])
df_sessions.index=df_grouped.index

In [148]:
df_sessions.loc[lambda x:(x.number_of_visits>1)&(x.maximal_clicks>1)].shape

(120, 4)